In [1]:
pip install soundfile pesq pystoi

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
from pathlib import Path
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchaudio


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
base_drive_path = Path("/content/drive/MyDrive/Mon_Projet")

In [5]:
import os
import pandas as pd
from pathlib import Path
from google.colab import drive

# 1. Monter Google Drive
drive.mount('/content/drive')

# --- CONFIGURATION DES CHEMINS ---
# Ajustez 'Mon_Projet' selon le nom de votre dossier dans Drive
base_drive_path = Path("/content/drive/MyDrive")


# Définition des dossiers spécifiques (comme dans votre exemple)
degraded_dir = base_drive_path / "train_packet_loss"
clean_dir = base_drive_path / "train_origin"

# 2. Lists to store the matched paths
dataset_records = []

# 3. Iterate through the "packet loss" folder
print(f"Scanning {degraded_dir}...")

files_found = 0
matches_found = 0

for degraded_file in degraded_dir.glob("*.wav"):
    files_found += 1
    filename = degraded_file.name  # Get just the name like "recording1.wav"

    # Construct the expected path for the clean file
    expected_clean_path = clean_dir / filename

    # 4. Check if the corresponding clean file exists
    if expected_clean_path.exists():
        # Append to our list
        dataset_records.append({
            "filename": filename,
            "packet_loss_path": str(degraded_file),
            "voice_origin_path": str(expected_clean_path)
        })
        matches_found += 1
    else:
        print(f"Warning: No match found for {filename} in clean directory.")

# 5. Create a DataFrame and save to CSV
df_train = pd.DataFrame(dataset_records)

# Save to CSV (common format for ML dataloaders)
output_csv = "audio_dataset_mapping_train.csv"
df_train.to_csv(output_csv, index=False)
# --- Summary ---
print("-" * 30)
print(f"Processing complete.")
print(f"Total files scanned: {files_found}")
print(f"Matched pairs found: {matches_found}")
print(f"Mapping saved to: {output_csv}")
print("-" * 30)

# Display the first few rows to verify
df_train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Scanning /content/drive/MyDrive/train_packet_loss...
------------------------------
Processing complete.
Total files scanned: 2128
Matched pairs found: 2118
Mapping saved to: audio_dataset_mapping_train.csv
------------------------------


,filename,packet_loss_path,voice_origin_path
0,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap22...,/content/drive/MyDrive/train_packet_loss/Jules...,/content/drive/MyDrive/train_origin/Jules_Vern...
1,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap19...,/content/drive/MyDrive/train_packet_loss/Jules...,/content/drive/MyDrive/train_origin/Jules_Vern...
2,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap12...,/content/drive/MyDrive/train_packet_loss/Jules...,/content/drive/MyDrive/train_origin/Jules_Vern...
3,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap01...,/content/drive/MyDrive/train_packet_loss/Jules...,/content/drive/MyDrive/train_origin/Jules_Vern...
4,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap02...,/content/drive/MyDrive/train_packet_loss/Jules...,/content/drive/MyDrive/train_origin/Jules_Vern...


In [6]:
import os
import pandas as pd
from pathlib import Path



# --- CONFIGURATION DES CHEMINS ---
# Ajustez 'Mon_Projet' selon le nom de votre dossier dans Drive
base_drive_path = Path("/content/drive/MyDrive")


# Définition des dossiers spécifiques (comme dans votre exemple)
degraded_dir = base_drive_path / "test_packet_loss"
clean_dir = base_drive_path / "test_origin"

# 2. Lists to store the matched paths
dataset_records = []

# 3. Iterate through the "packet loss" folder
print(f"Scanning {degraded_dir}...")

files_found = 0
matches_found = 0

for degraded_file in degraded_dir.glob("*.wav"):
    files_found += 1
    filename = degraded_file.name  # Get just the name like "recording1.wav"

    # Construct the expected path for the clean file
    expected_clean_path = clean_dir / filename

    # 4. Check if the corresponding clean file exists
    if expected_clean_path.exists():
        # Append to our list
        dataset_records.append({
            "filename": filename,
            "packet_loss_path": str(degraded_file),
            "voice_origin_path": str(expected_clean_path)
        })
        matches_found += 1
    else:
        print(f"Warning: No match found for {filename} in clean directory.")

# 5. Create a DataFrame and save to CSV
df_test = pd.DataFrame(dataset_records)

# Save to CSV (common format for ML dataloaders)
output_csv = "audio_dataset_mapping_test.csv"
df_test.to_csv(output_csv, index=False)
# --- Summary ---
print("-" * 30)
print(f"Processing complete.")
print(f"Total files scanned: {files_found}")
print(f"Matched pairs found: {matches_found}")
print(f"Mapping saved to: {output_csv}")
print("-" * 30)

# Display the first few rows to verify
df_test.head()

Scanning /content/drive/MyDrive/test_packet_loss...
------------------------------
Processing complete.
Total files scanned: 782
Matched pairs found: 782
Mapping saved to: audio_dataset_mapping_test.csv
------------------------------


,filename,packet_loss_path,voice_origin_path
0,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03...,/content/drive/MyDrive/test_packet_loss/Jules_...,/content/drive/MyDrive/test_origin/Jules_Verne...
1,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03...,/content/drive/MyDrive/test_packet_loss/Jules_...,/content/drive/MyDrive/test_origin/Jules_Verne...
2,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03...,/content/drive/MyDrive/test_packet_loss/Jules_...,/content/drive/MyDrive/test_origin/Jules_Verne...
3,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03...,/content/drive/MyDrive/test_packet_loss/Jules_...,/content/drive/MyDrive/test_origin/Jules_Verne...
4,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap20...,/content/drive/MyDrive/test_packet_loss/Jules_...,/content/drive/MyDrive/test_origin/Jules_Verne...


In [7]:
print(f"df_train shape: {df_train.shape}")
print(f"df_test shape: {df_test.shape}")

df_train shape: (2118, 3)
df_test shape: (782, 3)


In [8]:
# listen a .wav file
import IPython.display as ipd
ipd.Audio(df_train.iloc[0]['packet_loss_path'])

In [9]:
ipd.Audio(df_train.iloc[0]['voice_origin_path'])

# U-Net 1D

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        return self.conv(x)

class EncLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=8, stride=4):
        super().__init__()
        # Downsample with stride (Fast computation)
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride)
        self.relu = nn.ReLU(inplace=True)
        # 1x1 Conv + GLU (SOTA Gating mechanism)
        self.conv1x1 = nn.Conv1d(out_channels, 2 * out_channels, 1)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.conv1x1(x)
        # GLU: Splits channels in 2, controls information flow
        return F.glu(x, dim=1)

class DecLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=8, stride=4):
        super().__init__()
        # 1x1 Conv + GLU
        self.conv1x1 = nn.Conv1d(in_channels, 2 * in_channels, 1)
        self.glu = nn.GLU(dim=1)
        # Upsample with Transposed Conv (Learnable upsampling)
        self.trans_conv = nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride=stride)

    def forward(self, x):
        x = self.conv1x1(x)
        x = self.glu(x)
        return self.trans_conv(x)


class DemucsInpainter(nn.Module):
    def __init__(self, channels=48, depth=5):
        super().__init__()
        self.depth = depth
        self.encoder = nn.ModuleList()
        self.decoder = nn.ModuleList()

        current_c = 1
        for i in range(depth):
            out_c = channels * (2**i)
            self.encoder.append(EncLayer(current_c, out_c))
            current_c = out_c

        self.lstm = nn.LSTM(current_c, current_c, num_layers=2, bidirectional=True)
        self.lstm_proj = nn.Linear(current_c * 2, current_c)

        for i in reversed(range(depth)):
            out_c = channels * (2**(i-1)) if i > 0 else 1
            self.decoder.append(DecLayer(current_c, out_c))
            current_c = out_c

    def forward(self, x):
        skips = []

        # 1. Encode
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)

        # 2. Bottleneck
        x = x.permute(2, 0, 1)
        x, _ = self.lstm(x)
        x = self.lstm_proj(x)
        x = x.permute(1, 2, 0)

        # 3. Decode
        for i, layer in enumerate(self.decoder):
            skip = skips.pop()

            # --- 🛠️ FIX: Resize x to match skip BEFORE addition ---
            if x.shape[-1] != skip.shape[-1]:
                x = F.interpolate(x, size=skip.shape[-1], mode='nearest')
            # -----------------------------------------------------

            x = x + skip
            x = layer(x)

        return torch.tanh(x)

In [ ]:
def count_parameters(model):
    # Compte total des éléments (numel = number of elements)
    total_params = sum(p.numel() for p in model.parameters())

    # Compte seulement ceux qui ont requires_grad=True
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"--- Statistiques du Modèle ---")
    print(f"Total des paramètres : {total_params:,}") # La virgule ajoute un séparateur de milliers
    print(f"Paramètres entraînables : {trainable_params:,}")
    print(f"Poids estimé (float32) : {total_params * 4 / (1024**2):.2f} MB")

    return total_params

# --- Utilisation ---
model = WaveUNet(in_channels=1, out_channels=1)
count_parameters(model)

NameError: name 'WaveUNet' is not defined

In [11]:
import torch
import pandas as pd
import soundfile as sf
from pathlib import Path
from torch.utils.data import Dataset

def create_dataset_csv(degraded_dir_str, clean_dir_str, output_csv="audio_dataset_mapping.csv"):
    """
    Scans directories and creates a mapping CSV.
    """
    degraded_dir = Path(degraded_dir_str)
    clean_dir = Path(clean_dir_str)

    dataset_records = []
    matches_found = 0

    print(f"Scanning {degraded_dir}...")

    for degraded_file in degraded_dir.glob("*.wav"):
        filename = degraded_file.name
        expected_clean_path = clean_dir / filename

        if expected_clean_path.exists():
            dataset_records.append({
                "filename": filename,
                "packet_loss_path": str(degraded_file),
                "voice_origin_path": str(expected_clean_path)
            })
            matches_found += 1
        else:
            print(f"Warning: No match found for {filename}")

    df = pd.DataFrame(dataset_records)
    df.to_csv(output_csv, index=False)
    print(f"Mapping saved to: {output_csv} ({matches_found} pairs)")
    return df

class AudioDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (str or pd.DataFrame): Path to the csv file or the dataframe itself.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        if isinstance(csv_file, str):
            self.dataframe = pd.read_csv(csv_file)
        else:
            self.dataframe = csv_file

        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        degraded_path = self.dataframe.iloc[idx]['packet_loss_path']
        clean_path = self.dataframe.iloc[idx]['voice_origin_path']

        # 1. Load with soundfile
        degraded_np, _ = sf.read(degraded_path)
        clean_np, _ = sf.read(clean_path)

        # 2. Convert to Tensor (float32)
        degraded_waveform = torch.from_numpy(degraded_np).float()
        clean_waveform = torch.from_numpy(clean_np).float()

        # 3. Handle Channels (Ensure [C, T] shape)
        if degraded_waveform.ndim == 1:
            degraded_waveform = degraded_waveform.unsqueeze(0)
        else:
            degraded_waveform = degraded_waveform.t()

        if clean_waveform.ndim == 1:
            clean_waveform = clean_waveform.unsqueeze(0)
        else:
            clean_waveform = clean_waveform.t()

        if self.transform:
            degraded_waveform = self.transform(degraded_waveform)
            clean_waveform = self.transform(clean_waveform)

        return degraded_waveform, clean_waveform

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
import os

# --- 1. CONFIGURATION ---
CSV_PATH = "audio_dataset_mapping_train.csv"
CSV_PATH_TEST = "audio_dataset_mapping_test.csv"

BATCH_SIZE = 16
LEARNING_RATE = 1e-4
NUM_EPOCHS = 100
PATIENCE = 10     # For Early Stopping
NUM_WORKERS = 2   # Set to 0 if on Windows and you get errors

# Device Setup
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print(f"✅ Using CUDA: {torch.cuda.get_device_name(0)}")
else:
    DEVICE = torch.device("cpu")
    print("⚠️ CUDA not available. Using CPU.")

# --- 2. MODEL & DATASET PLACEHOLDERS ---
# ⚠️ IMPORTANT: Ensure your AudioDataset and DemucsInpainter classes
# are defined here or imported from another file.

# from my_dataset_file import AudioDataset
# from my_model_file import DemucsInpainter

# If they are in this file, paste them here:
# class AudioDataset(Dataset): ...
# class DemucsInpainter(nn.Module): ...

# --- 3. LOSS FUNCTION ---
class MultiScaleSTFTLoss(nn.Module):
    def __init__(self, fft_sizes=[1024, 2048, 512], hop_sizes=[120, 240, 50], win_lengths=[600, 1200, 240]):
        super().__init__()
        self.fft_sizes = fft_sizes
        self.hop_sizes = hop_sizes
        self.win_lengths = win_lengths

    def stft(self, x, fft_size, hop_size, win_length):
        x = x.squeeze(1)
        window = torch.hann_window(win_length, device=x.device)
        return torch.stft(x, n_fft=fft_size, hop_length=hop_size, win_length=win_length,
                          window=window, return_complex=True)

    def forward(self, x_pred, x_true):
        x_pred = x_pred.float()
        x_true = x_true.float()
        loss = 0.0

        for fft_size, hop_size, win_length in zip(self.fft_sizes, self.hop_sizes, self.win_lengths):
            x_pred_stft = self.stft(x_pred, fft_size, hop_size, win_length)
            x_true_stft = self.stft(x_true, fft_size, hop_size, win_length)

            x_pred_mag = torch.abs(x_pred_stft) + 1e-7
            x_true_mag = torch.abs(x_true_stft) + 1e-7

            sc_loss = torch.norm(x_true_mag - x_pred_mag, p="fro") / (torch.norm(x_true_mag, p="fro") + 1e-7)
            mag_loss = F.l1_loss(torch.log(x_true_mag), torch.log(x_pred_mag))

            loss += sc_loss + mag_loss

        return loss / len(self.fft_sizes)

# --- 4. MAIN TRAINING LOOP ---
def main():
    # A. Check Data
    if not os.path.exists(CSV_PATH):
        print(f"❌ Error: {CSV_PATH} not found. Please generate the CSV first.")
        return

    # B. Load Dataset
    print("Loading Datasets...")
    try:
        # ⚠️ Make sure AudioDataset is imported/defined!
        train_dataset = AudioDataset(CSV_PATH)
        val_dataset = AudioDataset(CSV_PATH_TEST)

        # Optional: Uncomment for quick debugging with small data
        # train_dataset = Subset(train_dataset, indices=list(range(100)))
        # val_dataset = Subset(val_dataset, indices=list(range(20)))
    except NameError:
        print("❌ Error: 'AudioDataset' class is not defined. Please import or paste it.")
        return

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        persistent_workers=True if NUM_WORKERS > 0 else False,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        persistent_workers=True if NUM_WORKERS > 0 else False,
        pin_memory=True
    )

    print(f"Data Loaded: {len(train_dataset)} Train, {len(val_dataset)} Val")

    # C. Model Setup
    print(f"Initializing Model on {DEVICE}...")
    try:
        # ⚠️ Make sure DemucsInpainter is imported/defined!
        model = DemucsInpainter(channels=48, depth=4).to(DEVICE)
    except NameError:
        print("❌ Error: 'DemucsInpainter' class is not defined. Please import or paste it.")
        return

    criterion_time = nn.L1Loss().to(DEVICE)
    criterion_freq = MultiScaleSTFTLoss().to(DEVICE)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    # --- Variables for Tracking ---
    best_val_loss = float('inf')  # Initialize this!
    early_stop_counter = 0        # Initialize this!

    print("🚀 Starting Training...")

    for epoch in range(NUM_EPOCHS):
        # ==========================
        #      TRAINING PHASE
        # ==========================
        model.train()
        running_loss = 0.0

        # TQDM for Training: Shows progress bar and live loss
        train_loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{NUM_EPOCHS}] [Train]", leave=True)

        for degraded_waveform, clean_waveform in train_loop:
            # Handle dimensions: (Batch, Time) -> (Batch, 1, Time)
            if degraded_waveform.dim() == 2: degraded_waveform = degraded_waveform.unsqueeze(1)
            if clean_waveform.dim() == 2: clean_waveform = clean_waveform.unsqueeze(1)

            degraded_waveform = degraded_waveform.to(DEVICE, non_blocking=True)
            clean_waveform = clean_waveform.to(DEVICE, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(degraded_waveform)

            # Crop to min length to avoid crashes
            min_len = min(outputs.shape[-1], clean_waveform.shape[-1])
            outputs = outputs[..., :min_len]
            clean_waveform = clean_waveform[..., :min_len]

            # Calculate Loss
            loss_time = criterion_time(outputs, clean_waveform)
            loss_freq = criterion_freq(outputs, clean_waveform)
            loss = loss_time + 2.5 * loss_freq

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss += loss.item()

            # Update the progress bar with the current loss
            train_loop.set_postfix(loss=loss.item())

        avg_train_loss = running_loss / len(train_loader)

        # ==========================
        #     VALIDATION PHASE
        # ==========================
        model.eval()
        val_loss = 0.0

        # TQDM for Validation (Optional, but helps to know it's not frozen)
        val_loop = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{NUM_EPOCHS}] [Val  ]", leave=False)

        with torch.no_grad():
            for degraded_waveform, clean_waveform in val_loop:
                if degraded_waveform.dim() == 2: degraded_waveform = degraded_waveform.unsqueeze(1)
                if clean_waveform.dim() == 2: clean_waveform = clean_waveform.unsqueeze(1)

                degraded_waveform = degraded_waveform.to(DEVICE, non_blocking=True)
                clean_waveform = clean_waveform.to(DEVICE, non_blocking=True)

                outputs = model(degraded_waveform)

                # Robust Shaping for Validation
                min_len = min(outputs.shape[-1], clean_waveform.shape[-1])
                outputs = outputs[..., :min_len]
                clean_waveform = clean_waveform[..., :min_len]

                loss_time = criterion_time(outputs, clean_waveform)
                loss_freq = criterion_freq(outputs, clean_waveform)

                val_loss += (loss_time + 2.5 * loss_freq).item()

        avg_val_loss = val_loss / len(val_loader)

        # Update Scheduler
        scheduler.step(avg_val_loss)

        # Print Epoch Summary
        # We use print here to keep a persistent log after the progress bars finish
        print(f"DONE Epoch {epoch+1} | Train Loss: {avg_train_loss:.5f} | Val Loss: {avg_val_loss:.5f}")

        # Save Best Model & Early Stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), "best_demucs_inpainter.pth")
            print("  💾 New best model saved!")
        else:
            early_stop_counter += 1
            print(f"  ⏳ No improvement. Early Stopping: {early_stop_counter}/{PATIENCE}")

        if early_stop_counter >= PATIENCE:
            print("🛑 Early stopping triggered.")
            break

if __name__ == "__main__":
    main()

✅ Using CUDA: Tesla T4
Loading Datasets...
Data Loaded: 2118 Train, 782 Val
Initializing Model on cuda...
🚀 Starting Training...


Epoch [1/100] [Train]: 100%|██████████| 133/133 [00:31<00:00,  4.19it/s, loss=6.15]


DONE Epoch 1 | Train Loss: 7.88281 | Val Loss: 6.10438
  💾 New best model saved!


Epoch [2/100] [Train]: 100%|██████████| 133/133 [00:31<00:00,  4.21it/s, loss=5.38]


DONE Epoch 2 | Train Loss: 5.80148 | Val Loss: 5.58775
  💾 New best model saved!


...

DONE Epoch 89 | Train Loss: 3.31964 | Val Loss: 3.19474
  ⏳ No improvement. Early Stopping: 8/10


Epoch [90/100] [Train]: 100%|██████████| 133/133 [00:30<00:00,  4.37it/s, loss=3.32]


DONE Epoch 90 | Train Loss: 3.31597 | Val Loss: 3.19312
  ⏳ No improvement. Early Stopping: 9/10


Epoch [91/100] [Train]: 100%|██████████| 133/133 [00:30<00:00,  4.36it/s, loss=2.86]


DONE Epoch 91 | Train Loss: 3.31284 | Val Loss: 3.21503
  ⏳ No improvement. Early Stopping: 10/10
🛑 Early stopping triggered.


In [ ]:
from torch.utils.data import Subset

train_dataset = AudioDataset(CSV_PATH)
train_dataset = Subset(train_dataset, indices=list(range(3)))
train_dataset

NameError: name 'CSV_PATH' is not defined

# Evaluation

In [12]:
import torch
import torchaudio
import os



def load_model(checkpoint_path, model_args=None):
    """
    Loads the model architecture and weights.
    """
    # Initialize the model structure
    # Note: You need to instantiate the model with the SAME arguments
    # (depth, channels, etc.) used during training.
    if model_args is None:
        model_args = {} # Add your model params here, e.g., {'num_channels': 1}

    model = DemucsInpainter(**model_args)

    # Load the weights
    if torch.cuda.is_available():
        device = torch.device('cuda')
        checkpoint = torch.load(checkpoint_path)
    else:
        device = torch.device('cpu')
        # map_location ensures we can load a GPU model on CPU
        checkpoint = torch.load(checkpoint_path, map_location=device)

    # Handle different saving methods (state_dict vs full checkpoint)
    if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'])
    elif isinstance(checkpoint, dict):
        model.load_state_dict(checkpoint)
    else:
        model = checkpoint # Fallback if the whole model object was saved

    model.to(device)
    model.eval() # Set to evaluation mode (disables Dropout, BatchNorm updates)
    return model, device

import librosa
import torch
import soundfile as sf # We use soundf_trainile to save the output

def predict(audio_path, output_path, model, device):
    # 1. Load using Librosa (loads as numpy array)
    # sr=None keeps the file's original sample rate
    waveform_np, sample_rate = librosa.load(audio_path, sr=None, mono=True)

    # 2. Convert to PyTorch Tensor
    # WaveUNet usually needs shape [1, 1, time_steps]
    input_tensor = torch.tensor(waveform_np).unsqueeze(0).unsqueeze(0).float().to(device)

    print(f"Processing {audio_path}...")

    # 3. Inference
    with torch.no_grad():
        output = model(input_tensor)

    # 4. Post-process
    if isinstance(output, dict):
        output = output['pred_audio']

    output_tensor = output.squeeze(0).squeeze(0).cpu().numpy()

    # 5. Save Output using soundf_trainile
    sf.write(output_path, output_tensor, sample_rate)
    print(f"Saved result to {output_path}")

In [ ]:
CHECKPOINT = "/content/best_demucs_inpainter.pth"
INPUT_FILE = "/content/drive/MyDrive/packet loss/test/Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap20_97_.wav"
OUTPUT_FILE = "output.wav"

model, device = load_model(CHECKPOINT)

predict(INPUT_FILE, OUTPUT_FILE, model, device)

NameError: name 'EncLayer' is not defined

In [ ]:
INPUT_FILE = "/content/drive/MyDrive/packet loss/test/Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_2_.wav"
OUTPUT_FILE = "output_chap03_2_.wav"
predict(INPUT_FILE, OUTPUT_FILE, model, device)

NameError: name 'model' is not defined

In [ ]:
degraded_dir

PosixPath('/content/drive/MyDrive/test_packet_loss')

In [13]:
import torch
import torch.nn as nn
import os
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
from pesq import pesq
from pystoi import stoi

# --- 1. MODEL DEFINITION (MUST MATCH TRAINING) ---
# ⚠️ IMPORTANT: Paste your DemucsInpainter class here or import it!

# ----------------------------------------------------


# --- 2. CONFIGURATION MODIFIÉE POUR COLAB ---
CHECKPOINT_PATH = "best_demucs_inpainter.pth"
# Mise à jour des chemins d'accès pour Colab
# Le fichier CSV n'est pas utilisé directement ici, mais le voici :
CSV_PATH_TEST = "/content/audio_dataset_mapping_test.csv"

# ⚠️ AJUSTER CES DEUX CHEMINS D'ACCÈS
# Assurez-vous que ces répertoires existent et contiennent vos fichiers .wav de test.
DEGRADED_DIR = base_drive_path / "test_packet_loss"
CLEAN_DIR = base_drive_path / "test_origin"

OUTPUT_DIR = "outputs_colab"
MODEL_ARGS = {'channels': 48, 'depth': 4}
# ----------------------------------------------------


# --- 3. HELPER FUNCTIONS ---

def load_model(checkpoint_path, model_args=None):
    """Loads the model architecture and weights."""
    if model_args is None: model_args = {}
    print(f"Loading model from {checkpoint_path}...")

    try:
        model = DemucsInpainter(**model_args)
    except NameError:
        print("❌ Error: 'DemucsInpainter' class not found. Please paste the class definition.")
        exit()

    if torch.cuda.is_available():
        device = torch.device('cuda')
        checkpoint = torch.load(checkpoint_path)
    else:
        device = torch.device('cpu')
        checkpoint = torch.load(checkpoint_path, map_location=device)

    if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'])
    elif isinstance(checkpoint, dict):
        model.load_state_dict(checkpoint)
    else:
        model = checkpoint

    model.to(device)
    model.eval()
    return model, device

def predict(audio_path, output_path, model, device):
    """Runs inference on a single audio file."""
    waveform_np, sample_rate = librosa.load(audio_path, sr=None, mono=True)
    input_tensor = torch.tensor(waveform_np).unsqueeze(0).unsqueeze(0).float().to(device)

    with torch.no_grad():
        output = model(input_tensor)

    if isinstance(output, dict):
        output = output['pred_audio']

    output_tensor = output.squeeze(0).squeeze(0).cpu().numpy()
    sf.write(output_path, output_tensor, sample_rate)
    return output_path

def evaluate_pair(reference_path, estimated_path):
    """Calculates PESQ and STOI for a pair of audio files."""
    TARGET_SR = 16000

    try:
        ref_audio, _ = librosa.load(reference_path, sr=TARGET_SR, mono=True)
        est_audio, _ = librosa.load(estimated_path, sr=TARGET_SR, mono=True)
    except Exception as e:
        # print(f"  ❌ Error loading files for metrics: {e}")
        return np.nan, np.nan

    min_len = min(len(ref_audio), len(est_audio))
    ref_audio = ref_audio[:min_len]
    est_audio = est_audio[:min_len]

    try:
        stoi_score = stoi(ref_audio, est_audio, TARGET_SR, extended=False)
    except:
        stoi_score = np.nan

    try:
        pesq_score = pesq(TARGET_SR, ref_audio, est_audio, 'wb')
    except:
        pesq_score = np.nan

    return pesq_score, stoi_score

# --- 4. MAIN EXECUTION ---

def main():
    if not os.path.exists(CHECKPOINT_PATH):
        print(f"❌ Error: Model file '{CHECKPOINT_PATH}' not found. Have you uploaded it to Colab?")
        return

    # 1. Load Model
    model, device = load_model(CHECKPOINT_PATH, MODEL_ARGS)
    print(f"✅ Model loaded on {device}")

    # 2. Prepare Directories
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    results = []

    if not os.path.exists(DEGRADED_DIR):
        print(f"❌ Error: Input directory '{DEGRADED_DIR}' not found. Please verify the Colab path.")
        return

    files = [f for f in os.listdir(DEGRADED_DIR) if f.lower().endswith('.wav')]
    print(f"📂 Found {len(files)} files to process in {DEGRADED_DIR}")

    # 3. Process Files
    for filename in files:
        degraded_path = os.path.join(DEGRADED_DIR, filename)
        reference_path = os.path.join(CLEAN_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, f"rec_{filename}")

        # A. Predict
        predict(degraded_path, output_path, model, device)

        # B. Evaluate
        if os.path.exists(reference_path):
            p_score, s_score = evaluate_pair(reference_path, output_path)
            results.append({
                "Filename": filename,
                "PESQ": p_score,
                "STOI": s_score
            })
            print(f"  --> {filename} | PESQ: {p_score:.2f} | STOI: {s_score:.2f}")
        else:
            print(f"  ⚠️ Processed {filename}, but NO reference found in {CLEAN_DIR} (Skipping metrics).")

    # 4. Save & Show Results
    if results:
        df = pd.DataFrame(results)

        avg_pesq = df["PESQ"].mean()
        avg_stoi = df["STOI"].mean()

        print("\n" + "="*30)
        print("       FINAL RESULTS       ")
        print("="*30)
        print(f"Average PESQ: {avg_pesq:.4f}")
        print(f"Average STOI: {avg_stoi:.4f}")
        print("="*30)

        csv_out = "evaluation_results_colab.csv"
        df.to_csv(csv_out, index=False)
        print(f"📝 Detailed results saved to {csv_out}")
    else:
        print("Done. No evaluation metrics generated (check if reference files exist).")

if __name__ == "__main__":
    main()

Loading model from best_demucs_inpainter.pth...
✅ Model loaded on cuda
📂 Found 782 files to process in /content/drive/MyDrive/test_packet_loss
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_49_.wav | PESQ: 1.20 | STOI: 0.75
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_65_.wav | PESQ: 1.26 | STOI: 0.75
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_4_.wav | PESQ: 1.18 | STOI: 0.70
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_59_.wav | PESQ: 1.22 | STOI: 0.75
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap20_73_.wav | PESQ: 1.14 | STOI: 0.65
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_6_.wav | PESQ: 1.13 | STOI: 0.71
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_150_.wav | PESQ: 1.10 | STOI: 0.65
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_74_.wav | PESQ: 1.27 | STOI: 0.66
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_8_.wav | PESQ: 1.13 | STOI: 0.63
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap17_144_.wav | PESQ: 1.15 |

# With the previous waveUnet

In [ ]:
# print average scores
avg_pesq = results_df_train['PESQ'].mean()
avg_stoi = results_df_train['STOI'].mean()
print(f"Average PESQ: {avg_pesq:.4f}")
print(f"Average STOI: {avg_stoi:.4f}")

Average PESQ: 1.1737
Average STOI: 0.6564
